In [1]:
from owlready2 import Thing, get_ontology, AllDisjoint, sync_reasoner_pellet, Imp, FunctionalProperty
from owlready2 import *

# Create a new ontology
onto = get_ontology("http://example.org/onto#")

# Define classes for the ontology based on the CSV file contents
with onto:
    class TVShow(Thing): pass
    class Episode(Thing): pass
    class User(Thing): pass
    class Comment(Thing): pass
    class Rating(Thing): pass
    class Emotion(Thing): pass

    # Define properties
    # Define relationships
    class posts(Comment >> User): pass
    class comments_on(Comment >> Episode): pass
    class follows(User >> TVShow): pass

    class has_watched(User >> Episode): pass
    class rated_by(Rating >> User): pass
    class rates(Rating >> Episode): pass
    class expressed_by(Emotion >> User): pass
    class evokes(Emotion >> Episode):  pass



* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation
